# DEW - Muscovite Phase Stability

Required python code to load the PhaseObjC library.

In [ ]:
from ctypes import cdll
from ctypes import util
import numpy as np
import ctypes
import matplotlib.pyplot as plt
import scipy.optimize as opt
from scipy.interpolate import interp1d
from rubicon.objc import ObjCClass, objc_method
import time
cdll.LoadLibrary(util.find_library('phaseobjc'))
%matplotlib inline

#### Create a python references to the DEWFluid, Andalusite, Kaolinite, Potassium Feldspar , Muscovite, Pyrophyllite and  Quartz class
#### Change convention for Berman properties
Use Helgeson (SUPCRT) convention of Gibbs free energy of formation rather than enthalpy of formation at Tr, Pr  
Do not implement quartz enthalpy correction that is used in rhyolite-MELTS

In [ ]:
DEWFluid = ObjCClass('DEWFluid').alloc().init()
Andalusite = ObjCClass('AndalusiteBerman').alloc().init()
Kaolinite = ObjCClass('KaoliniteBerman').alloc().init()
Kspar = ObjCClass('Potassium_FeldsparBerman').alloc().init()
Muscovite = ObjCClass('MuscoviteBerman').alloc().init()
Pyrophyllite = ObjCClass('PyrophylliteBerman').alloc().init()
Quartz = ObjCClass('QuartzBerman').alloc().init()

ObjCClass('QuartzBerman').enableGibbsFreeEnergyReferenceStateUsed()
ObjCClass('QuartzBerman').disableQuartzCorrectionUsed()

#### Check: Output phase names and formulae  

In [ ]:
print (DEWFluid.phaseName, DEWFluid.phaseFormula)
print (Andalusite.phaseName, Andalusite.phaseFormula)
print (Kaolinite.phaseName, Kaolinite.phaseFormula)
print (Kspar.phaseName, Kspar.phaseFormula)
print (Muscovite.phaseName, Muscovite.phaseFormula)
print (Pyrophyllite.phaseName, Pyrophyllite.phaseFormula)
print (Quartz.phaseName, Quartz.phaseFormula)

### Information about DEWFluid solution components ...

In [ ]:
nc = DEWFluid.numberOfSolutionComponents()
ns = DEWFluid.numberOfSolutionSpecies()
for i in range(0, ns):
    component = DEWFluid.componentAtIndex_(i)
    print (i, '\t', component.phaseName.ljust(20), '\t', component.phaseFormula.ljust(20), '\t', component.mw)

### Create a method that returns a vector of moles of endmember components ...
Note that internal to this function the vector is made by allocating a "c"-type pointer to a double precision one-dimensional array, and initialize the array to hold the moles of each component in the solution  
The default function call generates a mole vector correcsponding to a 1m KCl solution.

In [ ]:
def fillMolalityArray(mHAlO2=0.0, mSiO2=0.0, mHCl=1.0, mKOH=1.0, printme=False):
    m = (ctypes.c_double*nc)()
    ctypes.cast(m, ctypes.POINTER(ctypes.c_double))
    m[ 0] =  55.55 - mHAlO2 - min(mHCl, mKOH)
    m[ 1] =   0.0
    m[ 2] =   0.0
    m[ 3] =   0.0
    m[ 4] =   0.0
    m[ 5] =   0.0
    m[ 6] =   mHAlO2
    m[ 7] =   mSiO2
    m[ 8] =   0.0
    m[ 9] =   0.0
    m[10] =   mHCl
    m[11] =   mKOH
    m[12] =   0.0
    m[13] =   0.0
    m[14] =   0.0
    m[15] =   0.0
    m[16] =   0.0
    if printme == True:
        for i in range (0, nc):
            component = DEWFluid.componentAtIndex_(i)
            if m[i] > 0.0:
                print ('moles of (', component.phaseName.ljust(20), ') = ', m[i])
    return m

### Information about paramters that may be changed for HCl,aq solution component ,,,

### Information about solid phase parameters that may be changed ...
(kspar is demonstrated, muscovite, etc have the same parameters)

In [ ]:
nphases = Kspar.getNumberOfFreeParameters()
names = Kspar.getArrayOfNamesOfFreeParameters()
for i in range (0, nphases):
    name = names.objectAtIndex_(i)
    value = Kspar.getValueForParameterName_(name)
    units = Kspar.getUnitsForParameterName_(name)
    print (name, 'has value ', value, units)

### Load experimental data from: 
Sverjensky DA, Hemley JJ, D'Angelo WM (1991) Thermodynamic assessment of hydrothermal alkali feldspar-mica-aluminosilicate equilibria. *Geochimica et Cosmochimica Acta* 55, 989-1004.   

<div style="float:left">![picture](SverjenskyEtAl1991-Fig1a.png)</div>
  
m# | Assemblage | T °C | m<sub>t,K</sub> | log (m<sub>t,K</sub>/ m<sub>t,H</sub>)
--- | --- | --- | --- | ---
1 | KFs - Ms - Qtz | 300 | 1.00 | 3.50
2 | KFs - Ms - Qtz | 400 | 1.01 | 2.75
3 | KFs - Ms - Qtz | 500 | 1.01 | 1.95
4 | KFs - Ms - Qtz | 550 | 0.96 | 1.40
5 | Ms - And - Qtz | 400 | 0.98 | 1.60
6 | Ms - And - Qtz | 450 | 0.98 | 1.57
7 | Ms - And - Qtz | 500 | 0.97 | 1.47
8 | Ms - And - Qtz | 550 | 0.96 | 1.38
9 | Ms - Pyr - Qtz | 300 | 1.01 | 1.94
10 | Ms - Pyr - Qtz | 350 | 0.99 | 1.80
11 | Kaol - Ms - Qtz | 300 | 1.00 | 1.90 
12 | Kfs - And - Qtz | 600 | 0.81 | 0.63  

Fig. 1 Data plotted are at 100 MPa.  See table above.


Data are stored using Python tuples (t °C, p bars, m<sub>t,K</sub>, reaction #, log (m<sub>t,K</sub>/ m<sub>t,H</sub>))  
m is a list that indexes all of the tuples that hold the experimental data

In [ ]:
data = []
data.append((300.0, 1000.0, 1.00, 1, 3.5))
data.append((400.0, 1000.0, 1.00, 1, 2.75))
data.append((500.0, 1000.0, 1.00, 1, 1.95))
data.append((550.0, 1000.0, 0.96, 1, 1.40))
data.append((400.0, 1000.0, 0.98, 2, 1.60))
data.append((450.0, 1000.0, 0.98, 2, 1.57))
data.append((500.0, 1000.0, 0.97, 2, 1.47))
data.append((550.0, 1000.0, 0.96, 2, 1.38))
data.append((300.0, 1000.0, 1.00, 3, 1.94))
data.append((350.0, 1000.0, 0.99, 3, 1.80))
data.append((300.0, 1000.0, 1.00, 4, 1.90))
data.append((600.0, 1000.0, 0.81, 5, 0.63))
(t,p) = data[0][0:2]

### Define a function that performs fluid speciation using the DEW model

In [ ]:
def speciation(t, p, m, printme=False):
    muSpecies = DEWFluid.chemicalPotentialsOfSpeciesFromMolesOfComponents_andT_andP_(m, t, p)
    if printme == True:
        print ("{0:>20s} {1:>15s} {2:>15s} {3:>15s} {4:>15s}".format('species', 
                                                                     'activity', 
                                                                     'log10(activity)', 
                                                                     'mu', 
                                                                     'mu0'))
    result = {}
    for i in range (0, ns):
        if muSpecies.valueAtIndex_(i) != 0.0:
            pure = DEWFluid.componentAtIndex_(i)
            g = pure.getGibbsFreeEnergyFromT_andP_(t, p)
            result[DEWFluid.nameOfSolutionSpeciesAtIndex_(i)] = muSpecies.valueAtIndex_(i)
            if printme == True:
                print("{0:>20s} {1:15.6e} {2:15.6f}  {3:15.2f} {4:15.2f}".format(
                    DEWFluid.nameOfSolutionSpeciesAtIndex_(i), 
                    np.exp((muSpecies.valueAtIndex_(i)-g)/8.3143/t),
                    (muSpecies.valueAtIndex_(i)-g)/8.3143/t/np.log(10.0),
                    muSpecies.valueAtIndex_(i),
                    g))
    return result

### Define functions that return a vector of reaction energies for each of the five assemblages:

Assemblage 1 (Muscovite, Quartz, Kspar): 3 KAlSi<sub>3</sub>O<sub>8</sub> + 2 H<sup>+</sup> = KAl<sub>3</sub>Si<sub>3</sub>O<sub>10</sub>(OH)<sub>2</sub> + 2 K<sup>+</sup> + 6 SiO<sub>2</sub>  
Assemblage 2 (Muscovite, Quartz, Andalusite): 2 KAl<sub>3</sub>Si<sub>3</sub>O<sub>10</sub>(OH)<sub>2</sub> + 2 H<sup>+</sup> = 3 Al<sub>2</sub>SiO<sub>5</sub> + 3 SiO<sub>2</sub> + 2 K<sup>+</sup> + 3 H<sub>2</sub>O  
Assemblage 3 (Muscovite, Quartz, Pyrophyllite): 2 KAl<sub>3</sub>Si<sub>3</sub>O<sub>10</sub>(OH)<sub>2</sub> + 6 SiO<sub>2</sub> + 2 H<sup>+</sup> = 3 Al<sub>2</sub>Si<sub>4</sub>O<sub>10</sub>(OH)<sub>2</sub> + 2 K<sup>+</sup>  
Assemblage 4 (Muscovite, Quartz, Kaolinite): 2 KAl<sub>3</sub>Si<sub>3</sub>O<sub>10</sub>(OH)<sub>2</sub> + 2 H<sup>+</sup> + 3 H<sub>2</sub>O  = 3 Al<sub>2</sub>Si<sub>2</sub>O<sub>5</sub>(OH)<sub>4</sub> + 2 K<sup>+</sup>  
Assemblage 5 (Andalusite, Quartz, Kspar): 2 KAlSi<sub>3</sub>O<sub>8</sub> + 2 H<sup>+</sup> = Al<sub>2</sub>SiO<sub>5</sub> + 5 SiO<sub>2</sub> + 2 K<sup>+</sup> + H<sub>2</sub>O  

In [ ]:
# NOTE: Must fix debug flag in code to speed up by factor of 2-3
def reaction(x, t, p, assemblage, debug=False, returnDeltaGs=True,transformx=False):
    """
    Perform reaction calculation.
    
    optional argument transformx accepts compositional vector x in logit space to speed up energy optimization
    """
    if transformx:
        # x is really provided in logit-space and must be back-transformed to regular space
        x = transform_comp(x)
        
    m = fillMolalityArray(mHAlO2=x[0], mSiO2=x[1], mHCl=1.0, mKOH=x[2], printme=debug)
    if returnDeltaGs == True:
        muSp = speciation(t, p, m, printme=debug)
        deltaG1 = 0.0
        deltaG2 = 0.0
        deltaG3 = 0.0
        if assemblage == 1:  
            deltaG1 += muSp["K+"] + 3.0*muSp["Al+3"] + 3.0*muSp["SiO2,aq"] + 6.0*muSp["Water"]
            deltaG1 -= Muscovite.getGibbsFreeEnergyFromT_andP_(t, p) + 10.0*muSp["H+"] 
            deltaG2 += muSp["SiO2,aq"] - Quartz.getGibbsFreeEnergyFromT_andP_(t, p) 
            deltaG3 += muSp["K+"] + muSp["Al+3"] + 3.0*muSp["SiO2,aq"] + 2.0*muSp["Water"]
            deltaG3 -= Kspar.getGibbsFreeEnergyFromT_andP_(t, p) + 4.0*muSp["H+"] 
        elif assemblage == 2:
            deltaG1 += 2.0*muSp["Al+3"] + muSp["SiO2,aq"] + 3.0*muSp["Water"]
            deltaG1 -= Andalusite.getGibbsFreeEnergyFromT_andP_(t, p) + 6.0*muSp["H+"]
            deltaG2 += muSp["SiO2,aq"] - Quartz.getGibbsFreeEnergyFromT_andP_(t, p) 
            deltaG3 += muSp["K+"] + 3.0*muSp["Al+3"] + 3.0*muSp["SiO2,aq"] + 6.0*muSp["Water"]
            deltaG3 -= Muscovite.getGibbsFreeEnergyFromT_andP_(t, p) + 10.0*muSp["H+"] 
        elif assemblage == 3:
            deltaG1 += 2.0*muSp["Al+3"] + 4.0*muSp["SiO2,aq"] + 4.0*muSp["Water"]
            deltaG1 -= Pyrophyllite.getGibbsFreeEnergyFromT_andP_(t, p) + 6.0*muSp["H+"]
            deltaG2 += muSp["SiO2,aq"] - Quartz.getGibbsFreeEnergyFromT_andP_(t, p) 
            deltaG3 += muSp["K+"] + 3.0*muSp["Al+3"] + 3.0*muSp["SiO2,aq"] + 6.0*muSp["Water"]
            deltaG3 -= Muscovite.getGibbsFreeEnergyFromT_andP_(t, p) + 10.0*muSp["H+"]
        elif assemblage == 4:
            deltaG1 += 2.0*muSp["Al+3"] + 2.0*muSp["SiO2,aq"] + 5.0*muSp["Water"]
            deltaG1 -= Kaolinite.getGibbsFreeEnergyFromT_andP_(t, p) + 6.0*muSp["H+"]
            deltaG2 += muSp["SiO2,aq"] - Quartz.getGibbsFreeEnergyFromT_andP_(t, p) 
            deltaG3 += muSp["K+"] + 3.0*muSp["Al+3"] + 3.0*muSp["SiO2,aq"] + 6.0*muSp["Water"]
            deltaG3 -= Muscovite.getGibbsFreeEnergyFromT_andP_(t, p) + 10.0*muSp["H+"]
        elif assemblage == 5:
            deltaG1 += 2.0*muSp["Al+3"] + muSp["SiO2,aq"] + 3.0*muSp["Water"]
            deltaG1 -= Andalusite.getGibbsFreeEnergyFromT_andP_(t, p) + 6.0*muSp["H+"]
            deltaG2 += muSp["SiO2,aq"] - Quartz.getGibbsFreeEnergyFromT_andP_(t, p) 
            deltaG3 += muSp["K+"] + muSp["Al+3"] + 3.0*muSp["SiO2,aq"] + 2.0*muSp["Water"]
            deltaG3 -= Kspar.getGibbsFreeEnergyFromT_andP_(t, p) + 4.0*muSp["H+"]  
        return np.array([deltaG1, deltaG2, deltaG3])
    else:
        CapGam = 1000.0/18.01528
        moleFraction = DEWFluid.getSpeciesMoleFractionsForBulkComposition_aT_andP_(m, t, p)
        mHCl = moleFraction.objectForKey_("HCl,aq")*CapGam/moleFraction.objectForKey_("Water")
        mH   = moleFraction.objectForKey_("H+")*CapGam/moleFraction.objectForKey_("Water")
        mKCl = moleFraction.objectForKey_("KCl,aq")*CapGam/moleFraction.objectForKey_("Water")
        mKOH = moleFraction.objectForKey_("KOH,aq")*CapGam/moleFraction.objectForKey_("Water")
        mK   = moleFraction.objectForKey_("K+")*CapGam/moleFraction.objectForKey_("Water")
        if debug == True:
            print ("m t,K =", mKCl+mK+mKOH, m[11], "m t,H =", mHCl+mH)
        result = np.log((mKCl+mK)/(mHCl+mH))/np.log(10.0)
        return result
    
def transform_comp(xtrans):
    """
    Transform composition from logit-space to normal space
    """
    x = np.exp(xtrans)/(1+np.exp(xtrans))
    return x

def untransform_comp(x):
    """
    Transform composition from normal space to logit-space
    """
    xtrans = np.log(x/(1-x))
    return xtrans

### Calculate equilibrium curve for assemblage - kspar, muscovite, and quartz

In [ ]:
dim = 4 # dimension of the final vector storing all numbers
def generateAssemblage(xAssemblage, yAssemblage):
    guess = np.array([3.31199614e-06, 4.68942179e-02, 9.63461315e-01])
    guess_trans = untransform_comp(guess) 
    for i in range (dim):
        t = 550.0 - i*100.0
        
        reaction_trans = lambda x,t,p,assemblage,debug=False,returnDeltaGs=True: reaction(x,t,p,assemblage,transformx=True,debug=debug,returnDeltaGs=returnDeltaGs)
        args=(t+273.15, 1000.0, 1)
        
        result = opt.leastsq(reaction_trans, guess_trans,args=args,full_output=True)
        
        x_trans_fit = result[0]
        x_fit = transform_comp(x_trans_fit)
        dx_trans = x_trans_fit - guess_trans
        
        guess_trans = x_trans_fit + dx_trans
        
       
        print (t)
        print ('x_trans = ', x_trans_fit)
        print ('x = ',x_fit)
        ratio = reaction_trans(x_trans_fit, 873.15, 1000.0, 1, debug=False, returnDeltaGs=False)
        print (ratio)
        print('--------')
        xAssemblage1[i] = ratio
        yAssemblage1[i] = t

### Calculate equilibrium curve for assemblage 1 (muscovite, kspar, quartz)

#### 4.4.2 *with* corrections to the minerals

In [ ]:
param = -127235.44
component.setParameterName_tovalue_("deltaGibbsFreeEnergyOfFormationInTheReferenceState", param)
param = -5976740.0
Muscovite.setParameterName_tovalue_("delta H", param-1600.0*4.184)
param = -3970790.781 
Kspar.setParameterName_tovalue_("delta H", param-1600.0*4.184)

xAssemblage1 = np.zeros((dim,2))
yAssemblage1 = np.zeros(dim)
print ("Starting Assemblage ...")

t=time.time()
generateAssemblage(xAssemblage1,yAssemblage1)
print(time.time()-t)

In [ ]:
#%%timeit
param = Muscovite.getValueForParameterName_("deltaGibbsFreeEnergyOfFormationInTheReferenceState")
Muscovite.setParameterName_tovalue_("deltaGibbsFreeEnergyOfFormationInTheReferenceState", param - 1600.0*4.184)
param = Muscovite.getValueForParameterName_("delta H")
Muscovite.setParameterName_tovalue_("delta H", param - 1600.0*4.184)

param = Kspar.getValueForParameterName_("deltaGibbsFreeEnergyOfFormationInTheReferenceState")
Kspar.setParameterName_tovalue_("deltaGibbsFreeEnergyOfFormationInTheReferenceState", param-1600.0*4.184)
param = Kspar.getValueForParameterName_("delta H")
Kspar.setParameterName_tovalue_("delta H", param-1600.0*4.184)

xAssemblage2 = np.zeros((dim,2))
yAssemblage2 = np.zeros(dim)
print ("Starting Assemblage ...")
t=time.time()
generateAssemblage(xAssemblage2,yAssemblage2)
print ("... finally done!")
print(time.time()-t)

# NOTE: Cells below need updating from ENKI-web-version

### Plot up the experimental data and calculated curves ...

In [ ]:
xPlot1 = np.zeros(4)
yPlot1 = np.zeros(4)
xPlot1[0] = data[0][4]
yPlot1[0] = data[0][0]
xPlot1[1] = data[1][4]
yPlot1[1] = data[1][0]
xPlot1[2] = data[2][4]
yPlot1[2] = data[2][0]
xPlot1[3] = data[3][4]
yPlot1[3] = data[3][0]
plt.scatter(xPlot1, yPlot1, marker='o', color='r', label = 'KFs - Ms - Qtz' )
f = interp1d(yAssemblage1, xAssemblage1, kind='cubic')
ynew = np.linspace(yAssemblage1[0], yAssemblage1[7], num=100, endpoint=True)
plt.plot(f(ynew), ynew, color='r', label = 'calc KFs - Ms - Qtz' )

xPlot2 = np.zeros(4)
yPlot2 = np.zeros(4)
xPlot2[0] = data[4][4]
yPlot2[0] = data[4][0]
xPlot2[1] = data[5][4]
yPlot2[1] = data[5][0]
xPlot2[2] = data[6][4]
yPlot2[2] = data[6][0]
xPlot2[3] = data[7][4]
yPlot2[3] = data[7][0]
plt.scatter(xPlot2, yPlot2, marker='o', color='b', label = 'Ms - And - Qtz' )
f = interp1d(yAssemblage2, xAssemblage2, kind='cubic')
ynew = np.linspace(yAssemblage2[0], yAssemblage2[7], num=100, endpoint=True)
plt.plot(f(ynew), ynew, color='b', label = 'calc Ms - And - Qtz' )

xPlot3 = np.zeros(2)
yPlot3 = np.zeros(2)
xPlot3[0] = data[8][4]
yPlot3[0] = data[8][0]
xPlot3[1] = data[9][4]
yPlot3[1] = data[9][0]
plt.scatter(xPlot3, yPlot3, marker='o', color='g', label = 'Ms - Pyr - Qtz' )
f = interp1d(yAssemblage3, xAssemblage3, kind='cubic')
ynew = np.linspace(yAssemblage3[0], yAssemblage3[7], num=100, endpoint=True)
plt.plot(f(ynew), ynew, color='g', label = 'calc Ms - Pyr - Qtz' )

xPlot4 = np.zeros(1)
yPlot4 = np.zeros(1)
xPlot4[0] = data[10][4]
yPlot4[0] = data[10][0]
plt.scatter(xPlot4, yPlot4, marker='o', color='black', label = 'Kaol - Ms - Qtz' )
f = interp1d(yAssemblage4, xAssemblage4, kind='cubic')
ynew = np.linspace(yAssemblage4[0], yAssemblage4[7], num=100, endpoint=True)
plt.plot(f(ynew), ynew, color='black', label = 'calc Ms - kaol - Qtz' )

xPlot5 = np.zeros(1)
yPlot5 = np.zeros(1)
xPlot5[0] = data[11][4]
yPlot5[0] = data[11][0]
plt.scatter(xPlot5, yPlot5, marker='o', color='m', label = 'Kfs - And - Qtz' )
f = interp1d(yAssemblage5, xAssemblage5, kind='cubic')
ynew = np.linspace(yAssemblage5[0], yAssemblage5[7], num=100, endpoint=True)
plt.plot(f(ynew), ynew, color='m', label = 'calc Kfs - And - Qtz' )

plt.legend(loc='upper right')
plt.xlim(0.0, 8.0)
plt.ylim(200.0, 650.0)